### 자연어 데이터셋 전처리 <hr>
- Korpora 패키지 활용
- 데이터셋 : 한국어 혐오 데이터셋 korean_hate_speech

In [1]:
# 패키지 설치
# !pip install Korpora

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/57.8 kB ? eta -:--:--
   --------------------- ------------------ 30.7/57.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 57.8/57.8 kB 609.5 kB/s eta 0:00:00
   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---------------------------------------- 96.5/96.5 kB 1.8 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
# 패키지 설치 확인
import Korpora

Korpora.__version__

'0.2.0'

[1] 데이터 준비

In [7]:
# 데이터 다운로드
from Korpora import Korpora

# 데이터셋 파일명
data_file = 'korean_hate_speech'

# 다운로드 fetch
corpus = Korpora.fetch(data_file)

[korean hate speech] download unlabeled_comments_1.txt: 57.0MB [00:11, 4.93MB/s]                              
[korean hate speech] download unlabeled_comments_2.txt: 59.3MB [00:11, 5.03MB/s]                            
[korean hate speech] download unlabeled_comments_3.txt: 58.9MB [00:11, 4.93MB/s]                              
[korean hate speech] download unlabeled_comments_4.txt: 58.2MB [00:12, 4.77MB/s]                              
[korean hate speech] download unlabeled_comments_5.txt: 3.57MB [00:00, 3.61MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_1.txt: 44.7MB [00:08, 5.03MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_2.txt: 47.4MB [00:09, 4.91MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_3.txt: 48.5MB [00:09, 4.99MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_4.txt: 47.9MB [00:09, 5.04MB/

In [10]:
# 데이터 로딩
corpus = Korpora.load(data_file)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Authors :
        - Jihyung Moon* (inmoonlight@github)
        - Won Ik Cho* (warnikchow@github)
        - Junbum Lee (beomi@github)
        * equal contribution
    Repository : https://github.com/kocohub/korean-hate-speech
    References :
        - Moon, J., Cho, W. I., & Lee, J. (2020). BEEP! Korean Corpus of Online News
          Comments for Toxic Speech Detection. arXiv preprint arXiv:2005.12503.

    We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
    The data is comments from the Korean entertainment news aggregation platform.

    # License
    Creative Commons Attribution-ShareAlike 4.0 International License.
    Visit here for detail : https://creativec

In [14]:
# 데이터 확인
hateDS = corpus.train
hate_valDS = corpus.dev
hate_testDS = corpus.test
unlabelDS = corpus.unlabeled

# 1개 데이터 정보 추출
print(f'{hateDS[0].text}')
print(f'{hateDS[0].title}')
print(f'{hateDS[0].gender_bias}')
print(f'{hateDS[0].bias}')

print(unlabelDS[0])

(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....
"밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 [종합]"
False
others
SentencePair(text='"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"', pair='"[단독] 지드래곤♥이주연, 제주도 데이트…2018년 1호 커플 탄생"')


[2] 데이터 전처리 <hr>
- 주제에 따른 데이터 피처 & 라벨 선택 ===> 뉴스댓글 + 차별종류
- 형태소 분석기 설정

In [20]:
# 피처 데이터 추출
textData = corpus.get_all_texts()

with open ('../DATA/corpus.txt', 'w', encoding = 'utf-8') as f:
    for text in textData:
        f.write(text + '\n')


In [21]:
# !pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/991.7 kB ? eta -:--:--
   ---------------------------------------- 10.2/991.7 kB ? eta -:--:--
   --- ------------------------------------ 92.2/991.7 kB 1.1 MB/s eta 0:00:01
   ----------------------- ---------------- 573.4/991.7 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------  983.0/991.7 kB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 991.7/991.7 kB 5.2 MB/s eta 0:00:00


In [42]:
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.Train(input = '../DATA/corpus.txt',
                           model_prefix = 'text_bpe',
                           vocab_size = 8000,
                           model_type = 'bpe')

In [43]:
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load('text_bpe.model')

sentence = '안녕하세요, 토크나이저가 잘 학습되었군요!'
sentences = ['이렇게 입력값 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다.']

In [44]:
tokenized_sentence = tokenizer.encode_as_pieces(sentence)
tokenized_sentences = tokenizer.encode_as_pieces(sentences)

print('단일 문장 토큰화 :', tokenized_sentence)
print('여러 문장 토큰화 :', tokenized_sentences)

단일 문장 토큰화 : ['▁안녕', '하세요', ',', '▁토크', '나이', '저', '가', '▁잘', '▁학', '습', '되', '었', '군', '요', '!']
단일 문장 토큰화 : [['▁이렇게', '▁입', '력', '값', '▁리', '스트', '로', '▁받아', '서'], ['▁', '쉽', '게', '▁토크', '나이', '저', '를', '▁사용', '할', '▁수', '▁있', '답', '니다', '.']]


In [45]:
encoded_sentence = tokenizer.encode_as_ids(sentence)
encoded_sentences = tokenizer.encode_as_ids(sentences)

print('단일 문장 정수 인코딩 :', encoded_sentence)
print('여러 문장 정수 인코딩 :', encoded_sentences)

단일 문장 정수 인코딩 : [4598, 1073, 6681, 3614, 2850, 6901, 6711, 290, 816, 7105, 7102, 6929, 7149, 6812, 6799]
여러 문장 정수 인코딩 : [[1646, 116, 7000, 7541, 533, 314, 6709, 1561, 6726], [6677, 7525, 6806, 3614, 2850, 6901, 7124, 3689, 6886, 95, 146, 7195, 264, 6679]]


In [46]:
decode_ids = tokenizer.decode_ids(encoded_sentences)
decode_pieces = tokenizer.decode_pieces(encoded_sentences)

print('정수 인코딩에서 문장 변환:', decode_ids)
print('하위 단어 토큰에서 문장 변환:', decode_pieces)

정수 인코딩에서 문장 변환: ['이렇게 입력값 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다.']
하위 단어 토큰에서 문장 변환: ['이렇게 입력값 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다.']


In [47]:
tokenizer = SentencePieceProcessor()
tokenizer.load('text_bpe.model')

vocab = {idx : tokenizer.id_to_piece(idx) for idx in range(tokenizer.get_piece_size())}
print(list(vocab.items())[:5])
print('vocab size :', len(vocab))

[(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '""'), (4, '..')]
vocab size : 8000
